# UAS DL TASK 1 (GoEmotions)

## SETUP

In [1]:
!pip install transformers datasets evaluate accelerate scikit-learn -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 138.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


## Import

In [2]:
import os
os.environ["WANDB_DISABLED"] = "true"

import torch
import numpy as np
import evaluate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset

MODEL_CKPT = "distilbert-base-uncased"
BATCH_SIZE = 64
EPOCHS = 3

print(f"GPU Available: {torch.cuda.is_available()}")
print(f"Device Name: {torch.cuda.get_device_name(0)}")

GPU Available: True
Device Name: NVIDIA L4


## Load Data & Cek Labels

In [3]:
dataset = load_dataset("google-research-datasets/go_emotions")

labels_list = dataset["train"].features["labels"].feature.names
NUM_LABELS = len(labels_list)

print(f"Jumlah Label: {NUM_LABELS}")
print("5 Label Pertama:", labels_list[:5])
print("Contoh Raw Data:", dataset['train'][0])



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

simplified/train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

simplified/validation-00000-of-00001.par(…):   0%|          | 0.00/350k [00:00<?, ?B/s]

simplified/test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Jumlah Label: 28
5 Label Pertama: ['admiration', 'amusement', 'anger', 'annoyance', 'approval']
Contoh Raw Data: {'text': "My favourite food is anything I didn't have to cook myself.", 'labels': [27], 'id': 'eebbqej'}


## Preprocessing Multi-Hot

In [4]:
from datasets import Features, Sequence, Value

MODEL_CKPT = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_CKPT)

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CKPT,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

# 3. Preprocessing (DENGAN STRICT FLOAT)
def preprocess(examples):
    tokenized = tokenizer(examples["text"], truncation=True, padding=True, max_length=128)

    batch_labels = []
    for labels_ids in examples["labels"]:
        # Inisialisasi list float
        one_hot = [0.0] * NUM_LABELS
        for label_id in labels_ids:
            one_hot[label_id] = 1.0
        batch_labels.append(one_hot)

    tokenized["labels"] = batch_labels
    return tokenized

cols_to_remove = dataset["train"].column_names
tokenized_ds = dataset.map(preprocess, batched=True, remove_columns=cols_to_remove)

# Ganti tipe data ke float
new_features = tokenized_ds["train"].features.copy()
new_features["labels"] = Sequence(Value("float32"))

tokenized_ds = tokenized_ds.cast(new_features)
# VERIFIKASI TIPE DATA (Harus Float)
print("Contoh Label Pertama:", tokenized_ds['train'][0]['labels'][:5])
tokenized_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Cek tipe tensor
sample_label = tokenized_ds['train'][0]['labels']
print(f"Tipe Tensor Label: {sample_label.dtype}")
# HARUS: torch.float32. Jika int64/long -> Pasti Error.

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/43410 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5426 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5427 [00:00<?, ? examples/s]

Contoh Label Pertama: [0.0, 0.0, 0.0, 0.0, 0.0]
Tipe Tensor Label: torch.float32


## Model & Multi-Label Metrics

In [5]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_CKPT,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

# Metric F1 Macro untuk Multi Label
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits))
    y_pred = (probs > 0.3).int().numpy()
    y_true = labels.astype(int)

    f1_macro = f1_score(y_true, y_pred, average="macro")
    acc = accuracy_score(y_true, y_pred) # Exact Match

    return {"f1_macro": f1_macro, "accuracy": acc}

args = TrainingArguments(
    output_dir="./goemotions_bert_base",

    # Learning Rate Standar BERT
    learning_rate=3e-5,

    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,

    # 3 Epoch kurang untuk 28 label. 5 Epoch adalah titik ideal.
    num_train_epochs=5,

    # Regularization & Scheduler
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",

    eval_strategy="epoch",
    save_strategy="epoch",
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print(f"Starting Training with {MODEL_CKPT} (The Big Brain)...")
trainer.train()
trainer.save_model("./final_goemotions_bert")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/tmp/ipython-input-621105165.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting Training with bert-base-uncased (The Big Brain)...


Epoch,Training Loss,Validation Loss,F1 Macro,Accuracy
1,0.132600,0.101076,0.247898,0.423070
2,0.089200,0.086710,0.438003,0.449420
3,0.073000,0.084771,0.466924,0.459370
4,0.061400,0.087560,0.476428,0.445550
5,0.055200,0.089358,0.477579,0.445366


## Testing

In [7]:
model.eval()

labels_list = dataset["train"].features["labels"].feature.names
id2label_map = {i: label for i, label in enumerate(labels_list)}

# Kalimat uji coba dengan berbagai emosi
test_texts = [
    "I am so happy that I passed the exam with perfect score!", # Harusnya: joy/excitement
    "I lost my wallet and I feel terrible.",                   # Harusnya: sadness/disappointment
    "Get out of my room! I don't want to see you!",            # Harusnya: anger
    "Thank you so much for your help.",                        # Harusnya: gratitude
    "The food was okay, nothing special."                      # Harusnya: neutral
]

print("=== HASIL PREDIKSI GOEMOTIONS ===")
for text in test_texts:
    # 1. Tokenisasi
    inputs = tokenizer(text, return_tensors="pt").to("cuda")

    # 2. Prediksi
    with torch.no_grad():
        logits = model(**inputs).logits

    # 3. Proses Output (Sigmoid + Threshold 0.3)
    probs = torch.sigmoid(logits[0]) # Ambil batch pertama

    # Ambil index label yang probabilitasnya > 0.3
    # (x > 0.3).nonzero() mengembalikan index mana saja yang True
    predicted_indices = (probs > 0.3).nonzero(as_tuple=True)[0]

    # 4. Translate Index ke Nama Label
    predicted_labels = [id2label_map[idx.item()] for idx in predicted_indices]

    print(f"Teks: {text}")
    print(f"Prediksi: {predicted_labels}\n")

=== HASIL PREDIKSI GOEMOTIONS ===
Teks: I am so happy that I passed the exam with perfect score!
Prediksi: ['joy']

Teks: I lost my wallet and I feel terrible.
Prediksi: ['fear', 'sadness']

Teks: Get out of my room! I don't want to see you!
Prediksi: ['anger', 'annoyance']

Teks: Thank you so much for your help.
Prediksi: ['gratitude']

Teks: The food was okay, nothing special.
Prediksi: ['approval', 'neutral']

